# Импорт необходимых библиотек

In [1]:
# обработка данных
import pandas as pd

# добавление данных в базу
import psycopg2
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import psycopg2.extras

# работа с базой
import sqlite3

In [2]:
import config

# Первичная обработка данных

In [3]:
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [4]:
# Чтение и просмотр файла (выгружен с диска в формате .csv)
# Файл формата .xlsx читать сложнее, тем не менее, реализацию можно выполнить с помощью:
# project = pd.read_excel(r'<путь к файлу>')
# project = project.iloc[:<количество не пустых строк>, :<количество не путых столбцов, в нашем случае 8>]

project = pd.read_csv(r'Реестр проектов .xlsx - Проекты КБШ.csv')
project

,Реестр инновационных проектов,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Номер,Название проекта,Описание проекта,Информация о заявителе инновационного предложения,NaN,NaN,Функциональный заказчик,Текущий статус
1,п/п,NaN,NaN,Компания,Представитель компании (заявитель предложения),NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,Ф.И.О.,телефон,NaN,NaN
3,1,2,3,4,5,6,7,8
4,1,Система аналитики дислокации работников,Система позволяющая оповещать работников о приближении подвижного состава и информирования руководителя работ об удалении работников на безопасное расстояние от пути проследования подвижного состава;,ООО N146291,Новиков Л.Ф.,8(790)-197-50-64,П ДМВ Т НБТ,17.Внедрение
...,...,...,...,...,...,...,...,...
999,996,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,999,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# На случай, если остались полностью пустые колонки, можно попробовать этот вариант, но 
# использования метода .iloc предпочтительнее
# project.dropna(how='all', inplace=True)
# project.dropna(how='all', axis=1)

In [6]:
# Колонки в наборе данных считались плохо, поэтому переназовём их сами в соответствии с исходным файлом:
project.columns = ['Номер', 
                   'Название проекта',
                   'Описание проекта',
                   'Информация о заявителе инновационного предложения (Компания)',
                   'Информация о заявителе инновационного предложения (ФИО)',
                   'Информация о заявителе инновационного предложения (Телефон)',
                   'Функциональный заказчик',
                   'Текущий статус']

In [7]:
# Выведем общую информацию по набору данных (исключительно для лучшего понимания и анализа пустых значений)
project.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004 entries, 0 to 1003
Data columns (total 8 columns):
 #   Column                                                        Non-Null Count  Dtype 
---  ------                                                        --------------  ----- 
 0   Номер                                                         1003 non-null   object
 1   Название проекта                                              994 non-null    object
 2   Описание проекта                                              580 non-null    object
 3   Информация о заявителе инновационного предложения (Компания)  995 non-null    object
 4   Информация о заявителе инновационного предложения (ФИО)       995 non-null    object
 5   Информация о заявителе инновационного предложения (Телефон)   994 non-null    object
 6   Функциональный заказчик                                       994 non-null    object
 7   Текущий статус                                                994 non-null    o

In [8]:
# Кавычки и апострофы будут в дальнейшем мешать составлению базы данных, поэтому заменим их на похожие символы
# Сделать это необходимо во всех колонках таблицы, поэтому дополнительные отборы не нужны
# Можно было написать короче, но для наглядности и удобства проверки оставим две похожие строки:
project = project.replace("'", " ‘ ", regex = True)
project = project.replace('"', ' “ ', regex = True)


project = project.replace(',', ' , ', regex = True)
project = project.replace('\.', ' . ', regex = True)

project = project.replace('\(', ' ( ', regex = True)
project = project.replace('\)', ' ) ', regex = True)

project = project.replace('«', ' « ', regex = True)
project = project.replace('»', ' » ', regex = True)

project = project.replace('\-', ' - ', regex = True)


project = project.replace('  ', ' ', regex = True)



In [9]:
project

,Номер,Название проекта,Описание проекта,Информация о заявителе инновационного предложения (Компания),Информация о заявителе инновационного предложения (ФИО),Информация о заявителе инновационного предложения (Телефон),Функциональный заказчик,Текущий статус
0,Номер,Название проекта,Описание проекта,Информация о заявителе инновационного предложения,NaN,NaN,Функциональный заказчик,Текущий статус
1,п/п,NaN,NaN,Компания,Представитель компании ( заявитель предложения ),NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,Ф . И . О .,телефон,NaN,NaN
3,1,2,3,4,5,6,7,8
4,1,Система аналитики дислокации работников,Система позволяющая оповещать работников о приближении подвижного состава и информирования руководителя работ об удалении работников на безопасное расстояние от пути проследования подвижного состава;,ООО N146291,Новиков Л . Ф .,8 ( 790 ) - 197 - 50 - 64,П ДМВ Т НБТ,17 . Внедрение
...,...,...,...,...,...,...,...,...
999,996,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,999,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Окончательно обрезаем все ненужные и пустые строчки, методом, описанным в четвёртом блоке
project = project.iloc[4:996]

project = project.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)
project

,Номер,Название проекта,Описание проекта,Информация о заявителе инновационного предложения (Компания),Информация о заявителе инновационного предложения (ФИО),Информация о заявителе инновационного предложения (Телефон),Функциональный заказчик,Текущий статус
4,1,система аналитики дислокации работников,система позволяющая оповещать работников о приближении подвижного состава и информирования руководителя работ об удалении работников на безопасное расстояние от пути проследования подвижного состава;,ооо n146291,новиков л . ф .,8 ( 790 ) - 197 - 50 - 64,п дмв т нбт,17 . внедрение
5,2,антиобледенительные системы форс,"в отличие от большинства гидрофобных покрытий ( включающих в себя соединения на основе силикона ) , гидрофобные антиобледенительные композиции производятся на основе фторполимеров , что обеспечивает , помимо высокой гидрофобности , механическую прочность , долговечность покрытий , а также защиту поверхности от коррозии и образования льда .",ооо n966118,егоров х . д .,8 ( 979 ) - 553 - 80 - 67,дэз дпо,17 . внедрение
6,3,« осуществление опробования тормозов на основе технических средств без участия человека » сиптм - 395,разработка технологии применения сокращенного опробования тормозов в грузовых поездах при смене локомотива,ооо n179491,морозов г . и .,8 ( 405 ) - 945 - 58 - 27,т,17 . внедрение
7,4,система промывки систем отопления зданий « буча - к »,"оборудование для промывки систем отопления « буча - к » позволят эффективно произвести подготовку к отопительному сезону предотвратить аварийные отключения в отопительный период , снизить расходы на энергетические ресурсы .",ооо n947239,новиков л . к .,8 ( 568 ) - 594 - 55 - 28,дэз,17 . внедрение
8,5,"обслуживание робототехнических комплексов различного назначения , сенсорных и проводных систем и программного обеспечения","ао “ нпо “ андроидная техника “ специализируется на разработке , производстве , обслуживании робототехнических комплексов различного назначения , сенсорных и проводных систем и программного обеспечения",ооо n987114,иванов н . э .,8 ( 462 ) - 829 - 83 - 28,д,16 . адаптация решения для оао “ ржд “
...,...,...,...,...,...,...,...,...
991,988,антикоррозийная защита,антикоррозийная защита металлических конструкций,ооо n668573,евсеев э . м .,8 ( 779 ) - 646 - 89 - 14,п нтэ,3 . назначен менеджер
992,989,"вещество , предотвращающее отложение гололеда на контактных проводах железнодорожной контактной сети\nи разъединителях наружной установки \n","вещество , предотвращающее отложение гололеда на контактных проводах железнодорожной контактной сети\nи разъединителях наружной установки \n",ооо n380366,фомин ф . х .,8 ( 936 ) - 610 - 42 - 77,нтэ,11 . испытания
993,990,"биомикрогели для очистки воды от масел , жиров и нефтепродуктов","модимифированные полисахариды природного происхождения которые мгновенно капсулируют любые масла , жиры и нефтепродукты , отделяя их от воды",ооо n107438,ахмедшафин а . д .,8 ( 673 ) - 818 - 63 - 39,нцоп дтв,3 . назначен менеджер
994,991,удаленное управление козловыми контейнерными кранами,NaN,ооо n223643,никулин с . ю .,8 ( 294 ) - 955 - 66 - 15,дм,7 . экспертиза


In [11]:
project['function'] = list(map(lambda x: x.replace('  ', ' ').split(' '), project['Функциональный заказчик']))

In [12]:
project.shape

(992, 9)

# Реализация базы данных

## Добавление в базу данных таблицы с проектами

In [14]:
def connect_to_db():
    try:
        con = psycopg2.connect(
            database=config.DATABASE,
            user=config.USER,
            password=config.PASSWORD,
            host=config.HOST,
            port=config.PORT
        )
        con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        return con
    except (Exception, Error) as error:
        print("Ошибка при работе с PostgreSQL", error)


def connection_select(sql):
    con = connect_to_db()
    with con:
        with con.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
            cur.execute(sql)
            row = cur.fetchall()
            return row


def connection_insert_update(sql):
    con = connect_to_db()
    with con:
        with con.cursor() as cur:
            cur.execute(sql)


def push_request(number, year, theme, function):
    connection_insert_update(f'insert into requests (Номер, Год, Тема_открытого_запроса,'
                             f'Функциональный_заказчик) '
                             f'values (\'{number}\', \'{year}\', \'{theme}\', '
                             f'\'{function}\')'
                             )


def push_project(number, name, decsription, company, fio, telephone, function, status):
    connection_insert_update(f'insert into projects (Номер, Название_проекта, Описание_проекта, Компания, ФИО, '
                             f'телефон, Функциональный_заказчик, Текущий_статус)'
                             f'values (\'{number}\', \'{name}\', \'{decsription}\', \'{company}\', '
                             f'\'{fio}\', \'{telephone}\', \'{function}\', \'{status}\')'
                             )



In [16]:
project.shape

(992, 9)

In [17]:
# # Записываем данные в базу
for i in project.values.tolist():
    print(i[0])
    push_project(*i[:8])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## Добавление в базу данных таблицы с запросами¶

In [15]:
def get_clean_file(file_name):
    a = []
    f = open(file_name, 'r', encoding='UTF-8')
    for i in f.readlines():
        i = i.strip().strip('\ufeff').replace('\n', ' ')
        if i.count(';;') > 0:
            for j in range(i.count(';;')):
                i = i.replace(';;', ';')
        a.append(i.split(';'))
    return a

In [16]:
requests = get_clean_file("Перечень открытых запросов.csv")
# print(requests)
'''
Год	Тема открытого запроса	Функциональный заказчик
'''
df = pd.DataFrame(requests)
df = df.iloc[1:,:4]
df.columns = ['Номер', 'Год', 'Тема открытого запроса', 'Функциональный заказчик']

df

,Номер,Год,Тема открытого запроса,Функциональный заказчик
1,1,2019,Распознавание неисправностей и коммерческой непригодности подвижного состава в автоматическом режиме,КТЦФТО
2,2,2019,Защита населения от сверхнормативного акустического воздействия (вибрации) объектов железнодорожного транспорта,ДИ
3,3,2019,Определение содержания нефтепродуктов в сточных водах,ДТВ
4,4,2019,Поиск решений для предотвращения зажатия составителей поездов при производстве маневровой работы,Д
5,5,2019,Поиск инновационных решений в области промывки теплообменного оборудования объектов теплоснабжения,ДТВ
...,...,...,...,...
59,59,2021,"Оптимизация построения масштабных планов, профилей путей, измерения объемов грузов (не объявлен)",ДРП
60,60,2021,Применение современных систем ресурсосбережения локомотивов (не объявлен),Т
61,61,2021,Организация системы расширенной аналитики Холдинга (не объявлен),СИВЦ
62,62,2021,Применение автоматического заградительного устройства для прекращения движения автотранспортных средств при закрытии железнодорожного переезда для пропуска железнодорожного подвижного состава (не объявлен),П


In [17]:
df = df.replace("'", " ‘ ", regex = True)
df = df.replace('"', ' “ ', regex = True)


df = df.replace(',', ' , ', regex = True)
df = df.replace('\.', ' . ', regex = True)

df = df.replace('\(', ' ( ', regex = True)
df = df.replace('\)', ' ) ', regex = True)

df = df.replace('«', ' « ', regex = True)
df = df.replace('»', ' » ', regex = True)

df = df.replace('\-', ' - ', regex = True)


df = df.replace('  ', ' ', regex = True)
df

,Номер,Год,Тема открытого запроса,Функциональный заказчик
1,1,2019,Распознавание неисправностей и коммерческой непригодности подвижного состава в автоматическом режиме,КТЦФТО
2,2,2019,Защита населения от сверхнормативного акустического воздействия ( вибрации ) объектов железнодорожного транспорта,ДИ
3,3,2019,Определение содержания нефтепродуктов в сточных водах,ДТВ
4,4,2019,Поиск решений для предотвращения зажатия составителей поездов при производстве маневровой работы,Д
5,5,2019,Поиск инновационных решений в области промывки теплообменного оборудования объектов теплоснабжения,ДТВ
...,...,...,...,...
59,59,2021,"Оптимизация построения масштабных планов , профилей путей , измерения объемов грузов ( не объявлен )",ДРП
60,60,2021,Применение современных систем ресурсосбережения локомотивов ( не объявлен ),Т
61,61,2021,Организация системы расширенной аналитики Холдинга ( не объявлен ),СИВЦ
62,62,2021,Применение автоматического заградительного устройства для прекращения движения автотранспортных средств при закрытии железнодорожного переезда для пропуска железнодорожного подвижного состава ( не объявлен ),П


In [18]:
df = df.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)
df

,Номер,Год,Тема открытого запроса,Функциональный заказчик
1,1,2019,распознавание неисправностей и коммерческой непригодности подвижного состава в автоматическом режиме,ктцфто
2,2,2019,защита населения от сверхнормативного акустического воздействия ( вибрации ) объектов железнодорожного транспорта,ди
3,3,2019,определение содержания нефтепродуктов в сточных водах,дтв
4,4,2019,поиск решений для предотвращения зажатия составителей поездов при производстве маневровой работы,д
5,5,2019,поиск инновационных решений в области промывки теплообменного оборудования объектов теплоснабжения,дтв
...,...,...,...,...
59,59,2021,"оптимизация построения масштабных планов , профилей путей , измерения объемов грузов ( не объявлен )",дрп
60,60,2021,применение современных систем ресурсосбережения локомотивов ( не объявлен ),т
61,61,2021,организация системы расширенной аналитики холдинга ( не объявлен ),сивц
62,62,2021,применение автоматического заградительного устройства для прекращения движения автотранспортных средств при закрытии железнодорожного переезда для пропуска железнодорожного подвижного состава ( не объявлен ),п


In [61]:
db_req = df.values.tolist()
db_req

[['1',
  '2019',
  'распознавание неисправностей и коммерческой непригодности подвижного состава в автоматическом режиме',
  'ктцфто'],
 ['2',
  '2019',
  'защита населения от сверхнормативного акустического воздействия ( вибрации ) объектов железнодорожного транспорта',
  'ди'],
 ['3', '2019', 'определение содержания нефтепродуктов в сточных водах', 'дтв'],
 ['4',
  '2019',
  'поиск решений для предотвращения зажатия составителей поездов при производстве маневровой работы',
  'д'],
 ['5',
  '2019',
  'поиск инновационных решений в области промывки теплообменного оборудования объектов теплоснабжения',
  'дтв'],
 ['6',
  '2019',
  'поиск инновационных решений в области достоверности учета топливно - энергетических ресурсов',
  'дмс'],
 ['7',
  '2019',
  'поиск инновационных решений в области утилизации органических отходов ii - v классов опасности с получением экологически безопасных продуктов',
  'нбт'],
 ['8',
  '2019',
  'поиск инновационных решений в области автоматизации процесса у

In [67]:
# Записываем данные в базу
# for i in db_req:
#     print(i)
#     push_request(*i[:4])

['1', '2019', 'распознавание неисправностей и коммерческой непригодности подвижного состава в автоматическом режиме', 'ктцфто']
['2', '2019', 'защита населения от сверхнормативного акустического воздействия ( вибрации ) объектов железнодорожного транспорта', 'ди']
['3', '2019', 'определение содержания нефтепродуктов в сточных водах', 'дтв']
['4', '2019', 'поиск решений для предотвращения зажатия составителей поездов при производстве маневровой работы', 'д']
['5', '2019', 'поиск инновационных решений в области промывки теплообменного оборудования объектов теплоснабжения', 'дтв']
['6', '2019', 'поиск инновационных решений в области достоверности учета топливно - энергетических ресурсов', 'дмс']
['7', '2019', 'поиск инновационных решений в области утилизации органических отходов ii - v классов опасности с получением экологически безопасных продуктов', 'нбт']
['8', '2019', 'поиск инновационных решений в области автоматизации процесса учета негабаритных мест и формирования маршрута пропуска

# Реализация критериев отбора

In [50]:
# Ввод запроса по ключевым словам
def input_words_request():
    request = input().lower().split(' ')
    exception_words = ['VR', 'AR']
    
    for word in request:
        if len(word) <=3 and word not in exception_words:
            request.remove(word)

    request = list(map(lambda x: x[:-2] if len(x)>3 else x, request))
    return request


In [51]:
# отбор по заказчику
def selection_by_functional_requester(word):
    word = word.lower()
    return pd.DataFrame(connection_select(f"""select r.id as "номер", r.Тема_открытого_запроса, p.id, p."Название_проекта", p."Описание_проекта", r."Функциональный_заказчик", p."Функциональный_заказчик" as "Функциональный_заказчик_проекта" from requests as r join projects as p on r."Функциональный_заказчик" = p."Функциональный_заказчик" or not position(' ' || r."Функциональный_заказчик" || ' ' in p."Функциональный_заказчик") = 0 WHERE '{word}' = r."Функциональный_заказчик";"""))

In [52]:
# Совпадения по ключевым словам:
def selection_by_keywords(words):
    for word in words:
        word = word.lower()
        yield pd.DataFrame(connection_select(f"""SELECT p.id, p."Описание_проекта", p."Название_проекта" FROM projects AS p WHERE NOT POSITION(' ' || '{word}' IN p."Описание_проекта" || p."Название_проекта") = 0;""")).values.tolist()
        

In [140]:
!pip install openpyxl

     |████████████████████████████████| 242 kB 833 kB/s            


In [22]:
import openpyxl

In [59]:
def rating_by_words_projects(name='dict1'):
    a = input_words_request()

    cur_dict = dict()

    asav = selection_by_keywords(a)
    for i in asav:
        for number in i:
            # number = number[0]
            if number[0] not in cur_dict:
                cur_dict[number[0]] = {"Индекс релевантности":1,"Название проекта":number[1], "Описание проекта":number[2]}
            else:
                cur_dict[number[0]]["Индекс релевантности"] += 1
                
#     print(*[str(i)+str(j) for i,j in cur_dict.items()], sep='\n')
    cur_dict = dict(sorted(cur_dict.items(), key=lambda item: item[0], reverse=True))
 

    df = pd.DataFrame(data=cur_dict)
    
    df = (df.T)
#     print (*df, sep='\n')
    df.to_excel(name+'.xlsx', index_label='ID в базе данных')

    return cur_dict

In [78]:
rating_by_words_projects()

Система аналитика
3{'Индекс релевантности': 1, 'Название проекта': 'использование указанного светильника в системе интеллектуального освещения пассажирских платформ позволит обеспечить рациональное использование электроэнергии . ', 'Описание проекта': 'разработка компактного диммируемого светодиодного светильника с двумя режимами работы'}
7{'Индекс релевантности': 1, 'Название проекта': 'установка на маневровый локомотив информационно - измерительной системы для контроля геометрических параметров стрелочных переводов с передачей данных в режиме онлайн', 'Описание проекта': 'разработка информационно - измерительной системы ( иис ) для контроля геометрических параметров стрелочных переводов и ее установка на маневровый локомотив'}
10{'Индекс релевантности': 2, 'Название проекта': 'программным обеспечением по оценке и прогнозированию расхода топлива специальным самоходным подвижным составом . аналитика позволяет выявлять факты чрезмерного потребления для исследования и устранения причин .

{975: {'Индекс релевантности': 1,
  'Название проекта': 'использование интеллектуальной системы oko для идентификации лиц с целью организации беключевого доступа сотрудников самарского ивц в помещения с ограниченным доступом ( например в машинный зал , зал единой смены самарского ивц и т . п . ) с возможностью фиксации времени входа и выхода и хранения информации в электронном журнале . ',
  'Описание проекта': 'использование интеллектуальной системы для идентификации лиц с целью организации беключевого доступа'},
 974: {'Индекс релевантности': 1,
  'Название проекта': 'использование системы « око » , обеспечивающей дополнительной безопасностью необходимый объект с возможностью идентификации личности по сетчатки глаза;',
  'Описание проекта': 'интеллектуальная система распознавания и бесключевого доступа человека'},
 973: {'Индекс релевантности': 1,
  'Название проекта': 'предложение oko system возможно использовать в здании поста эц ( например , ст . самара ) для разграничения доступа

In [88]:
res_2 = selection_by_functional_requester("Д")

res_2[['id', 'Функциональный_заказчик_проекта']]

,id,Функциональный_заказчик_проекта
0,11,т д нс
1,22,ш д
2,82,джв д нтэ
3,119,т д
4,149,сивц нки т д ди
5,150,сивц нки т д ди


In [54]:
def find_by_func_req(cur_dict, requester):
    df = pd.DataFrame(cur_dict).T
    res_2 = selection_by_functional_requester(requester)[['id', 'Функциональный_заказчик_проекта']]

    
#     df_2 = df.set_index('ID в базе данных').join(res_2.set_index('ID в базе данных'),on = 'ID в базе данных', how = 'inner')
#     print(df.columns)
    df.reset_index(level = 0, inplace = True)
    df_2 = pd.merge(df, res_2, how = 'inner', left_on='id',right_on='id')
    return df_2

In [113]:
ans = (set(res) & set(res_2))
ans

set()

In [ ]:
def selection_by_companies(company_name):
    

# Поиск по Запросам

In [55]:
# Совпадения по ключевым словам:
def selection_by_keywords_requests(words):
    for word in words:
        yield pd.DataFrame(connection_select(f"""SELECT r.id, r."Год", r."Тема_открытого_запроса" FROM requests AS r WHERE NOT POSITION(' ' || '{word}' IN r."Тема_открытого_запроса") = 0;""")).values.tolist()
        

In [56]:
def rating_by_words_requests(name='dict1'):
    a = input_words_request()
    cur_dict = dict()

    asav = selection_by_keywords_requests(a)
    for i in asav:
        for number in i:
            # number = number[0]
            if number[0] not in cur_dict:
                cur_dict[number[0]] = {"Индекс релевантности":1,"Год":number[1], "Тема":number[2]}
            else:
                cur_dict[number[0]]["Индекс релевантности"] += 1
                
#     print(*[str(i)+str(j) for i,j in cur_dict.items()], sep='\n')
    cur_dict = dict(sorted(cur_dict.items(), key=lambda item: item[0], reverse=True))
 

    df = pd.DataFrame(data=cur_dict)
    
    df = (df.T)
#     print (*df, sep='\n')
    df.to_excel(name+'.xlsx', index_label='ID в базе данных')

    return cur_dict

In [89]:
rating_by_words_requests('Система')

Система
24{'Индекс релевантности': 1, 'Год': '2020', 'Тема': 'создание нового поколения бортовых систем безопасности с использованием компьютерных технологий с элементами искусственного интеллекта'}
32{'Индекс релевантности': 1, 'Год': '2020', 'Тема': 'автоматизация деятельности химико - технологической лаборатории и ее интеграции с системами управления ( отменен ) '}
36{'Индекс релевантности': 1, 'Год': '2020', 'Тема': 'разработка системы/оборудования неразрушающего контроля конструкций пассажирских платформ посредством удаленного сбора информации с созданием 3d проекции и с визуализацией дефектов'}
39{'Индекс релевантности': 1, 'Год': '2020', 'Тема': 'интегрированная система электронного помощника при выполнении технологического процесса'}
42{'Индекс релевантности': 1, 'Год': '2021', 'Тема': 'мобильные системы взвешивания железнодорожного подвижного состава'}
44{'Индекс релевантности': 1, 'Год': '2021', 'Тема': 'применение компактных , масштабируемых и безопасных систем бесперебойног

{63: {'Индекс релевантности': 1,
  'Год': '2021',
  'Тема': 'мониторинг состояния систем пожарной безопасности ( не объявлен ) '},
 61: {'Индекс релевантности': 1,
  'Год': '2021',
  'Тема': 'организация системы расширенной аналитики холдинга ( не объявлен ) '},
 60: {'Индекс релевантности': 1,
  'Год': '2021',
  'Тема': 'применение современных систем ресурсосбережения локомотивов ( не объявлен ) '},
 52: {'Индекс релевантности': 1,
  'Год': '2021',
  'Тема': 'установка автоматизированной вентиляционной системы в химико - технических лабораториях'},
 44: {'Индекс релевантности': 1,
  'Год': '2021',
  'Тема': 'применение компактных , масштабируемых и безопасных систем бесперебойного питания на топливных элементах'},
 42: {'Индекс релевантности': 1,
  'Год': '2021',
  'Тема': 'мобильные системы взвешивания железнодорожного подвижного состава'},
 39: {'Индекс релевантности': 1,
  'Год': '2020',
  'Тема': 'интегрированная система электронного помощника при выполнении технологического проце

# Консольное решение

In [57]:
def menu():
    print('Напишите, что вы хотите сделать')
    print('1 - Поиск проекта по запроса')
    print('2 - Поиск запроса по проекту')
    a = input()
    
    if a == '1':
        name = input('Введите название эксель файла: ')
        print('Введите поисковый запрос: ')
        cur_dict = rating_by_words_projects(name)
        print(f'Ваш ответ сохранен в файле {name+".xlsx"}')
        
#         reply = input('Хотите ли вы отфильтровать по Функциональным заказчикам?\n1 - да\n2 - нет\n')
#         if reply == '1':
#             requester = input('Укажите одного Функционального заказчика: ')
#             print(find_by_func_req(cur_dict, requester))
        
#         reply = input('Хотите ли вы отфильтровать по компаниям?\n1 - да\n2 - нет\n')
#         if reply == '1':
#             pass
        
    elif a == '2':
        name = input('Введите название эксель файла: ')
        print('Введите поисковый запрос: ')
        rating_by_words_requests(name)
        name = name+".xlsx"
        print(f'Ваш ответ сохранен в файле {name}')
        
#         reply = input('Хотите ли вы отфильтровать по Функциональным заказчикам?\n1 - да\n2 - нет\n')
#         if reply == '1':
#             pass
        
        

        
    

In [61]:
menu()

Напишите, что вы хотите сделать
1 - Поиск проекта по запроса
2 - Поиск запроса по проекту
1
Введите название эксель файла: технология
Введите поисковый запрос: 
технология
Ваш ответ сохранен в файле технология.xlsx
Хотите ли вы отфильтровать по Функциональным заказчикам?
1 - да
2 - нет
2
Хотите ли вы отфильтровать по компаниям?
1 - да
2 - нет
2


###### Черновики для реализации поиковых запросов (может быть кому-нибудь потом пригодится работа со строками, конвертация списков и т.д.)

In [ ]:
pd.DataFrame(connection_select(f"""SELECT p.id, p."Описание_проекта", p."Название_проекта" FROM projects AS p WHERE NOT POSITION(' ' || 'Систе' IN p."Описание_проекта" || p."Название_проекта") = 0;"""))

In [18]:
# def dict_project_database_creation(data):
#     """
#     Функция принимает на вход базу данных с проектами, в которой есть колонки "Название проекта", "Описание проекта":
#     Функиция возвращает словарь слов со второстепенным значением и добавляет в исходные данные колонки со словарями (тегами)
#     и их общим количеством в документе
#     """
#         
#     data['Описание проекта тэги'] = list(map(lambda x: str(x).strip().split(' '), data['Описание проекта']))
#     data['Название проекта тэги'] = list(map(lambda x: str(x).strip().split(' '), data['Название проекта']))
#     
#     return data

In [19]:
# cur_data = dict_project_database_creation(project.copy())# 
# cur_data['intersect'] = list(map(lambda x,y: list(set(x)&set(y)),cur_data["Описание проекта тэги"], cur_data["Название проекта тэги"]))
# cur_data

In [20]:
# def get_clean_file(file_name):
#     a = []
#     f = open(file_name, 'r', encoding='UTF-8')
#     for i in f.readlines():
#         i = i.strip().strip('\ufeff').replace('\n', ' ')
#         if i.count(';;') > 0:
#             for j in range(i.count(';;')):
#                 i = i.replace(';;', ';')
#         a.append(i.split(';'))
#     return a
# 
# 
# def write_file(file_name, array):
#     f = open(file_name, 'w')
#     for i in array:
#         a = ';'.join(i)
#         print(a)
#         f.write(a)
#         f.write('\n')
# 
# 
# def make_dict(array):
#     cur_ar_dict = []
#     first = array[0]
#     for i_array in array[1:]:
#         cur_dict = {}
#         for i, j in zip(i_array, first):
#             cur_dict[j] = i
#         cur_ar_dict.append(cur_dict)
#     return cur_ar_dict
#
# для самопроверки:
# project = get_clean_file('Реестр проектов.csv')
# write_file('project.csv', project.csv)
# g = get_clean_file('project2.csv')
# print(*g, sep='\n')
# k = open('ans','w')
# print(*make_dict(project), sep='\n', file=k)
#
# requests = get_clean_file("Перечень открытых запросов.csv")
# dict_requests = make_dict(requests)
# print(*dict_requests, sep='\n')